In [1]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import scipy

import collections
import datetime
import time

import geopandas as gpd

import numpy as np
 
from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from sklearn import linear_model
import statsmodels.api as sm

from linearmodels import PanelOLS, FamaMacBeth
from scipy import stats

import itertools


## Grab Data

In [14]:
os.getcwd()

'/Users/brianreed/Documents/supplyChain/extremes/extremesAnalysisCode'

In [4]:
# goodsData = pd.read_csv("../goodsData_suppliers.csv").drop(columns = {'Unnamed: 0'})
# goodsData = pd.read_csv("../goodsData_igData.csv").drop(columns = {'Unnamed: 0'})
goodsData = pd.read_csv("../utilitiesData_igData.csv").drop(columns = {'Unnamed: 0'})

# goodsData = pd.read_csv("../goodsData.csv").drop(columns = {'Unnamed: 0'})
'''goodsData = goodsData[~goodsData.revenueChange.isna() & 
                     ~goodsData.costChange.isna()]
'''
# goodsData['precip5Days_annualquant_1x5Yrs'] = 1*(goodsData['precip5Days_annualquant_1x5Yrs'] > 0)

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


'goodsData = goodsData[~goodsData.revenueChange.isna() & \n                     ~goodsData.costChange.isna()]\n'

# goodsData

Let's grab a list with all the relevant variable names. Just keep the core names and we'll grab the lags as we go through it.

Look at the full combinations of variables and whatnot, if we'd like.

In [24]:
'''
print(goodsData.columns[0:50])

cutoffVars  = ['0.95','1xQtr','1xYr','1x5Qtrs','1x5Yrs'] # ,'1x10Qtrs','1x10Yrs'
weatherVars = ['precip_','temp_','precip5Days_','temp5Days_']
statVars    = ['annualquant_','zipquant_','zipQuarterquant_']

outcomeVars = ['lnCost','lnRev','lnCostNormd','lnRevNormd']
# ['incomeChange', 'revenueChange', 'costChange'],'lnCost','lnInc','lnRev']
# ['lnCost','lnRev','lnCostNormd','lnRevNormd']


for weatherVar in weatherVars:
    for statVar in statVars:
        for cutoffVar in cutoffVars:
            indVar = weatherVar + statVar + cutoffVar
            
            for outcomeVar in outcomeVars:
                print(indVar,"~", outcomeVar)'''

'\nprint(goodsData.columns[0:50])\n\ncutoffVars  = [\'0.95\',\'1xQtr\',\'1xYr\',\'1x5Qtrs\',\'1x5Yrs\'] # ,\'1x10Qtrs\',\'1x10Yrs\'\nweatherVars = [\'precip_\',\'temp_\',\'precip5Days_\',\'temp5Days_\']\nstatVars    = [\'annualquant_\',\'zipquant_\',\'zipQuarterquant_\']\n\noutcomeVars = [\'lnCost\',\'lnRev\',\'lnCostNormd\',\'lnRevNormd\']\n# [\'incomeChange\', \'revenueChange\', \'costChange\'],\'lnCost\',\'lnInc\',\'lnRev\']\n# [\'lnCost\',\'lnRev\',\'lnCostNormd\',\'lnRevNormd\']\n\n\nfor weatherVar in weatherVars:\n    for statVar in statVars:\n        for cutoffVar in cutoffVars:\n            indVar = weatherVar + statVar + cutoffVar\n            \n            for outcomeVar in outcomeVars:\n                print(indVar,"~", outcomeVar)'

Focus on the 5 day variables now (we'll later need to edit the loops above to account for this as well, but we can do this for now.) Make them 0/1.

# Direct Effects
Look at the effects on the suppliers when they're affected directly.

The below gives us the full, clustered standard errors.

In [26]:
cutoffVarsYr = ['0.95']  #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_'] #, 'temp5Days_', 'precip5Days_'] # , 'precip_']#, , ] #[,]
statVarsYr   = ['zipquant_', 'ffquant_', 'zipQuarterquant_', 'indQuarterquant_'] #  , , ]  #,'zipQuarterquant_']
outcomeVars  = ['lnRevNormd', 'lnCostNormd']# , 'lnRev', 'lnCost', 'revenueChange', 'costChange']

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna()] # & ~goodsData.lnCostNormd.isna()]


start = time.time()

results = pd.DataFrame()

i = 0
for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:
        for statVar in statVarsYr:                     
            for cutoffVar in cutoffVarsYr:
                i = i + 1
                indVar = weatherVar + statVar + cutoffVar
                
                
                print(outcomeVar, "~", indVar)


                # find: concurrent ; or lagged supplier data
                X = goodsData.loc[:,((goodsData.columns.str.contains(indVar)) | 
                                                (goodsData.columns.str.contains('indQtr_')) |
                                                (goodsData.columns.str.contains('gvkey_')))]#  | 
                                                # (goodsData.columns.str.contains('firmQtr_')) |
                                                #(goodsData.columns.str.contains('ageQtr_')) |
                                                #(goodsData.columns.str.contains('sizeQtr_')) |
                                                #(goodsData.columns.str.contains('profitQtr_')))]
                
                
                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]
                
                
                model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                
                # results.loc[i,'ind'] = ind


                results.loc[i,'outcomeVar'] = outcomeVar
                results.loc[i,'weatherVar'] = weatherVar
                results.loc[i,'statVar']    = statVar
                results.loc[i,'cutoffVar']  = cutoffVar
                # results.loc[i,'firmVar']    = firmVar


                results.loc[i,'lag0']       = coeff[0]
                results.loc[i,'lag1']       = coeff[1]
                results.loc[i,'lag2']       = coeff[2]
                results.loc[i,'lag3']       = coeff[3]
                results.loc[i,'lag4']       = coeff[4]

                results.loc[i,'pval0']      = pvals[0]
                results.loc[i,'pval1']      = pvals[1]
                results.loc[i,'pval2']      = pvals[2]
                results.loc[i,'pval3']      = pvals[3]
                results.loc[i,'pval4']      = pvals[4]

                
                results.to_csv("../../data/utilitiesResults_rightInds_noCtrls.csv")
                
                print( time.time() - start)

lnRevNormd ~ precip_zipquant_0.95
0.9773828983306885
lnRevNormd ~ precip_ffquant_0.95
2.520321846008301
lnRevNormd ~ precip_zipQuarterquant_0.95
3.8356997966766357
lnRevNormd ~ precip_indQuarterquant_0.95
5.075920820236206
lnRevNormd ~ temp_zipquant_0.95
6.713193893432617
lnRevNormd ~ temp_ffquant_0.95
7.9631028175354
lnRevNormd ~ temp_zipQuarterquant_0.95
9.13026475906372
lnRevNormd ~ temp_indQuarterquant_0.95
10.570092916488647
lnCostNormd ~ precip_zipquant_0.95
11.576021909713745
lnCostNormd ~ precip_ffquant_0.95
12.521055936813354
lnCostNormd ~ precip_zipQuarterquant_0.95
13.596194982528687
lnCostNormd ~ precip_indQuarterquant_0.95
14.867467880249023
lnCostNormd ~ temp_zipquant_0.95
15.853435754776001
lnCostNormd ~ temp_ffquant_0.95
16.845340728759766
lnCostNormd ~ temp_zipQuarterquant_0.95
17.825721740722656
lnCostNormd ~ temp_indQuarterquant_0.95
19.082683801651


In [15]:
print(ind)

35


In [54]:
results.to_csv("../../data/utilitiesResults_rightInds.csv")

## Employment-Wtd Weather
Run the regressions using the emp-wtd data.

In [37]:
cutoffVar   = '0.95'
weatherVar  = 'precip_'
statVar  = 'zipquant_'
outcomeVar  = 'lnRevNormd'

indVar = weatherVar + statVar + cutoffVar


goodsData.columns[goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('empWt_')] 

Index(['empWt_precip_zipquant_0.95', 'empWt_lag1_precip_zipquant_0.95',
       'empWt_lag2_precip_zipquant_0.95', 'empWt_lag3_precip_zipquant_0.95',
       'empWt_lag4_precip_zipquant_0.95'],
      dtype='object')

In [40]:
cutoffVar   = '0.95'
weatherVar  = 'precip_'
statVarYr  = 'zipquant_'
outcomeVar  = 'lnRevNormd'

ind = 2


##################
filename = '../../data/companyData/igData_ind' + str(ind) + '.csv'           
goodsData = pd.read_csv(filename).drop(columns = {'Unnamed: 0'})


indVar = weatherVar + statVar + cutoffVar


print(outcomeVar, "~", indVar)


# find: concurrent ; or lagged supplier data
X = goodsData.loc[:,((goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('empWt_')) | 
                                (goodsData.columns.str.contains('indQtr_')) |
                                (goodsData.columns.str.contains('gvkey_'))  | 
                                (goodsData.columns.str.contains('ageTercile_')) |
                                (goodsData.columns.str.contains('sizeTercile_')) |
                                (goodsData.columns.str.contains('profitTercile_')))]


print(X.columns)

firms = goodsData['gvkey']


y = goodsData[outcomeVar]


model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]

print(model.summary())

lnRevNormd ~ precip_zipquant_0.95
Index(['empWt_precip_zipquant_0.95', 'empWt_lag1_precip_zipquant_0.95',
       'empWt_lag2_precip_zipquant_0.95', 'empWt_lag3_precip_zipquant_0.95',
       'empWt_lag4_precip_zipquant_0.95', 'gvkey_2606', 'gvkey_4579',
       'gvkey_4809', 'gvkey_5568', 'gvkey_5597', 'gvkey_6375', 'gvkey_6573',
       'gvkey_6574', 'gvkey_7696', 'gvkey_8336', 'gvkey_8582', 'gvkey_9411',
       'gvkey_9774', 'gvkey_10345', 'gvkey_10598', 'gvkey_10609',
       'gvkey_10793', 'gvkey_12825', 'gvkey_12978', 'gvkey_13456',
       'gvkey_13540', 'gvkey_14503', 'gvkey_15087', 'gvkey_21506',
       'gvkey_21798', 'gvkey_24437', 'gvkey_24755', 'gvkey_24990',
       'gvkey_28877', 'gvkey_29517', 'gvkey_29609', 'gvkey_29942',
       'gvkey_30651', 'gvkey_34447', 'gvkey_62655', 'gvkey_64102',
       'gvkey_65238', 'gvkey_65570', 'gvkey_66016', 'gvkey_109084',
       'gvkey_114415', 'gvkey_118261', 'gvkey_123434', 'gvkey_142953',
       'gvkey_144435', 'gvkey_147308', 'gvkey_148276'

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1873: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 69, but rank is 9
  'rank is %d' % (J, J_), ValueWarning)


In [46]:
cutoffVarsYr = ['0.95'] # , '1x5Qtrs', '1x5Yrs'] # '1x5Qtrs',
weatherVars  = ['precip_', 'temp_']        #, 'temp5Days_', 'precip5Days_'] # , 'precip_']#, , ] #[,]
statVarsYr   = ['annualquant_','zipquant_','zipQuarterquant_']
outcomeVars  = ['lnRevNormd', 'lnCostNormd', 'lnRev', 'lnCost', 'revenueChange', 'costChange']

industries = range(1,44)

start = time.time()

results = pd.DataFrame()

i = 0



for ind in industries:
    print('##########################################################')
    print(ind)
    filename = '../../data/companyData/igData_ind' + str(ind) + '.csv'           
    goodsData = pd.read_csv(filename).drop(columns = {'Unnamed: 0'})

    
    for outcomeVar in outcomeVars:
        for weatherVar in weatherVars:
            for statVar in statVarsYr:                     
                for cutoffVar in cutoffVarsYr:
                    
                    i = i + 1
                    
                    
                    
                    '''goodsData = goodsData[~goodsData.lnRev.isna() & 
                                         ~goodsData.lnCost.isna() & 
                                         ~goodsData.revenueChange.isna() & 
                                         ~goodsData.costChange.isna()]'''
                    
                    
                    indVar = weatherVar + statVar + cutoffVar


                    print(outcomeVar, "~", indVar)


                    # find: concurrent ; or lagged supplier data
                    X = goodsData.loc[:,((goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('empWt_')) | 
                                                    (goodsData.columns.str.contains('indQtr_')) |
                                                    (goodsData.columns.str.contains('gvkey_'))  | 
                                                    (goodsData.columns.str.contains('ageTercile_')) |
                                                    (goodsData.columns.str.contains('sizeTercile_')) |
                                                    (goodsData.columns.str.contains('profitTercile_')))]


                    firms = goodsData['gvkey']


                    y = goodsData[outcomeVar]


                    model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                    pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                    coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                    print(pvals)
                    # results.loc[i,'ind'] = ind

                    results.loc[i,'industry'] = ind

                    results.loc[i,'outcomeVar'] = outcomeVar
                    results.loc[i,'weatherVar'] = weatherVar
                    results.loc[i,'statVar']    = statVar
                    results.loc[i,'cutoffVar']  = cutoffVar
                    # results.loc[i,'firmVar']    = firmVar


                    results.loc[i,'lag0']       = coeff[0]
                    results.loc[i,'lag1']       = coeff[1]
                    results.loc[i,'lag2']       = coeff[2]
                    results.loc[i,'lag3']       = coeff[3]
                    results.loc[i,'lag4']       = coeff[4]

                    results.loc[i,'pval0']      = pvals[0]
                    results.loc[i,'pval1']      = pvals[1]
                    results.loc[i,'pval2']      = pvals[2]
                    results.loc[i,'pval3']      = pvals[3]
                    results.loc[i,'pval4']      = pvals[4]


                    results.to_csv("../../results_byInds_withControls_empWts.csv")

                    print( time.time() - start)

##########################################################
1
lnRevNormd ~ precip_annualquant_0.95
empWt_precip_annualquant_0.95         3.708800e-02
empWt_lag1_precip_annualquant_0.95    1.028504e-01
empWt_lag2_precip_annualquant_0.95    7.561711e-02
empWt_lag3_precip_annualquant_0.95    1.665973e-01
empWt_lag4_precip_annualquant_0.95    6.688806e-03
gvkey_2812                            4.005091e-03
gvkey_3801                            2.432106e-04
gvkey_10390                           3.981033e-07
gvkey_10884                           9.012602e-01
gvkey_24539                           8.967151e-01
dtype: float64
0.06932520866394043
lnRevNormd ~ precip_zipquant_0.95
empWt_precip_zipquant_0.95         6.828390e-01
empWt_lag1_precip_zipquant_0.95    3.049496e-01
empWt_lag2_precip_zipquant_0.95    9.580910e-02
empWt_lag3_precip_zipquant_0.95    3.265245e-01
empWt_lag4_precip_zipquant_0.95    4.016560e-01
gvkey_2812                         1.083919e-05
gvkey_3801                         

empWt_precip_zipQuarterquant_0.95         0.039939
empWt_lag1_precip_zipQuarterquant_0.95    0.539789
empWt_lag2_precip_zipQuarterquant_0.95    0.752080
empWt_lag3_precip_zipQuarterquant_0.95    0.174145
empWt_lag4_precip_zipQuarterquant_0.95    0.410875
gvkey_2812                                0.416559
gvkey_3801                                0.294335
gvkey_10390                               0.802428
gvkey_10884                               0.347593
gvkey_24539                               0.023536
dtype: float64
0.4094831943511963
costChange ~ temp_annualquant_0.95
empWt_temp_annualquant_0.95         0.160984
empWt_lag1_temp_annualquant_0.95    0.585909
empWt_lag2_temp_annualquant_0.95    0.513340
empWt_lag3_temp_annualquant_0.95    0.666319
empWt_lag4_temp_annualquant_0.95    0.170315
gvkey_2812                          0.696004
gvkey_3801                          0.100876
gvkey_10390                         0.818692
gvkey_10884                         0.720269
gvkey_24539     

2.002115249633789
lnRev ~ precip_annualquant_0.95
empWt_precip_annualquant_0.95         0.078781
empWt_lag1_precip_annualquant_0.95    0.007067
empWt_lag2_precip_annualquant_0.95    0.008466
empWt_lag3_precip_annualquant_0.95    0.191317
empWt_lag4_precip_annualquant_0.95    0.010499
gvkey_2606                            0.001872
gvkey_4579                            0.019370
gvkey_4809                            0.784909
gvkey_5568                            0.438323
gvkey_5597                            0.894923
dtype: float64
2.282820224761963
lnRev ~ precip_zipquant_0.95
empWt_precip_zipquant_0.95         3.453988e-02
empWt_lag1_precip_zipquant_0.95    7.429372e-03
empWt_lag2_precip_zipquant_0.95    2.139471e-02
empWt_lag3_precip_zipquant_0.95    8.867025e-02
empWt_lag4_precip_zipquant_0.95    2.100579e-02
gvkey_2606                         1.947923e-17
gvkey_4579                         1.255434e-02
gvkey_4809                         5.409100e-01
gvkey_5568                        

empWt_temp_annualquant_0.95         0.375449
empWt_lag1_temp_annualquant_0.95    0.129137
empWt_lag2_temp_annualquant_0.95    0.151766
empWt_lag3_temp_annualquant_0.95    0.276875
empWt_lag4_temp_annualquant_0.95    0.836857
gvkey_2606                          0.000359
gvkey_4579                          0.620889
gvkey_4809                          0.663508
gvkey_5568                          0.001883
gvkey_5597                          0.138912
dtype: float64
4.892345190048218
revenueChange ~ temp_zipquant_0.95
empWt_temp_zipquant_0.95         0.288573
empWt_lag1_temp_zipquant_0.95    0.261859
empWt_lag2_temp_zipquant_0.95    0.228921
empWt_lag3_temp_zipquant_0.95    0.200125
empWt_lag4_temp_zipquant_0.95    0.029877
gvkey_2606                       0.804376
gvkey_4579                       0.682928
gvkey_4809                       0.965114
gvkey_5568                       0.000988
gvkey_5597                       0.127176
dtype: float64
4.9910359382629395
revenueChange ~ temp_zipQuar

6.048088073730469
lnRev ~ precip_zipQuarterquant_0.95
empWt_precip_zipQuarterquant_0.95         7.490374e-02
empWt_lag1_precip_zipQuarterquant_0.95    5.145482e-02
empWt_lag2_precip_zipQuarterquant_0.95    4.260439e-03
empWt_lag3_precip_zipQuarterquant_0.95    2.254375e-02
empWt_lag4_precip_zipQuarterquant_0.95    1.858145e-02
gvkey_3144                                5.731073e-07
gvkey_12761                               7.777686e-06
gvkey_18231                               2.664735e-08
gvkey_18397                               1.081283e-08
gvkey_106254                              2.925888e-02
dtype: float64
6.064785003662109
lnRev ~ temp_annualquant_0.95
empWt_temp_annualquant_0.95         0.752388
empWt_lag1_temp_annualquant_0.95    0.514447
empWt_lag2_temp_annualquant_0.95    0.334347
empWt_lag3_temp_annualquant_0.95    0.990723
empWt_lag4_temp_annualquant_0.95    0.533715
gvkey_3144                          0.000019
gvkey_12761                         0.012459
gvkey_18231       

...

lnRevNormd ~ precip_annualquant_0.95
empWt_precip_annualquant_0.95         0.772878
empWt_lag1_precip_annualquant_0.95    0.758554
empWt_lag2_precip_annualquant_0.95    0.604473
empWt_lag3_precip_annualquant_0.95    0.644299
empWt_lag4_precip_annualquant_0.95    0.819773
gvkey_1121                            0.073908
gvkey_1659                            0.748430
gvkey_1773                            0.928575
gvkey_1919                            0.887475
gvkey_2101                            0.656011
dtype: float64
526.6573619842529
lnRevNormd ~ precip_zipquant_0.95
empWt_precip_zipquant_0.95         0.742282
empWt_lag1_precip_zipquant_0.95    0.637066
empWt_lag2_precip_zipquant_0.95    0.367995
empWt_lag3_precip_zipquant_0.95    0.844084
empWt_lag4_precip_zipquant_0.95    0.898813
gvkey_1121                         0.008949
gvkey_1659                         0.662307
gvkey_1773                         0.909691
gvkey_1919                         0.870575
gvkey_2101                    

empWt_temp_zipquant_0.95         0.199911
empWt_lag1_temp_zipquant_0.95    0.066094
empWt_lag2_temp_zipquant_0.95    0.096067
empWt_lag3_temp_zipquant_0.95    0.078339
empWt_lag4_temp_zipquant_0.95    0.022104
gvkey_1121                       0.000011
gvkey_1659                       0.001262
gvkey_1773                       0.000011
gvkey_1919                       0.000001
gvkey_2101                       0.005233
dtype: float64
532.5088851451874
lnRev ~ temp_zipQuarterquant_0.95
empWt_temp_zipQuarterquant_0.95         0.168265
empWt_lag1_temp_zipQuarterquant_0.95    0.265970
empWt_lag2_temp_zipQuarterquant_0.95    0.066461
empWt_lag3_temp_zipQuarterquant_0.95    0.057507
empWt_lag4_temp_zipQuarterquant_0.95    0.009751
gvkey_1121                              0.000024
gvkey_1659                              0.001190
gvkey_1773                              0.000008
gvkey_1919                              0.000002
gvkey_2101                              0.005254
dtype: float64
532.8413

empWt_precip_zipquant_0.95         0.661926
empWt_lag1_precip_zipquant_0.95    0.760432
empWt_lag2_precip_zipquant_0.95    0.010699
empWt_lag3_precip_zipquant_0.95    0.010575
empWt_lag4_precip_zipquant_0.95    0.060455
gvkey_1121                         0.054602
gvkey_1659                         0.034260
gvkey_1773                         0.000135
gvkey_1919                         0.000152
gvkey_2101                         0.068363
dtype: float64
538.4572370052338
costChange ~ precip_zipQuarterquant_0.95
empWt_precip_zipQuarterquant_0.95         0.902275
empWt_lag1_precip_zipQuarterquant_0.95    0.715752
empWt_lag2_precip_zipQuarterquant_0.95    0.586175
empWt_lag3_precip_zipQuarterquant_0.95    0.089263
empWt_lag4_precip_zipQuarterquant_0.95    0.209967
gvkey_1121                                0.988840
gvkey_1659                                0.114649
gvkey_1773                                0.001322
gvkey_1919                                0.000445
gvkey_2101                 

empWt_precip_annualquant_0.95         7.162063e-04
empWt_lag1_precip_annualquant_0.95    3.964979e-03
empWt_lag2_precip_annualquant_0.95    2.126543e-03
empWt_lag3_precip_annualquant_0.95    4.445704e-03
empWt_lag4_precip_annualquant_0.95    4.318201e-03
gvkey_5523                            4.155287e-02
gvkey_8815                            1.485481e-06
gvkey_9063                            1.500149e-14
gvkey_9284                            1.280851e-05
gvkey_9359                            1.047237e-26
dtype: float64
541.0361340045929
lnRev ~ precip_zipquant_0.95
empWt_precip_zipquant_0.95         1.351488e-02
empWt_lag1_precip_zipquant_0.95    3.333804e-02
empWt_lag2_precip_zipquant_0.95    2.839843e-02
empWt_lag3_precip_zipquant_0.95    3.200495e-02
empWt_lag4_precip_zipquant_0.95    3.088932e-02
gvkey_5523                         5.936954e-04
gvkey_8815                         1.888459e-13
gvkey_9063                         3.779533e-20
gvkey_9284                         1.070052e

542.1004371643066
revenueChange ~ temp_annualquant_0.95
empWt_temp_annualquant_0.95         0.323076
empWt_lag1_temp_annualquant_0.95    0.038317
empWt_lag2_temp_annualquant_0.95    0.976739
empWt_lag3_temp_annualquant_0.95    0.011625
empWt_lag4_temp_annualquant_0.95    0.454630
gvkey_5523                          0.003971
gvkey_8815                          0.000101
gvkey_9063                          0.000130
gvkey_9284                          0.000124
gvkey_9359                          0.001879
dtype: float64
542.1707401275635
revenueChange ~ temp_zipquant_0.95
empWt_temp_zipquant_0.95         0.991704
empWt_lag1_temp_zipquant_0.95    0.475522
empWt_lag2_temp_zipquant_0.95    0.765507
empWt_lag3_temp_zipquant_0.95    0.262508
empWt_lag4_temp_zipquant_0.95    0.741701
gvkey_5523                       0.016599
gvkey_8815                       0.000103
gvkey_9063                       0.000053
gvkey_9284                       0.000135
gvkey_9359                       0.000986
dtype:

## Industry-Specific
Go through every famafrench industry and run the regressions above.

In [45]:
cutoffVarsYr = ['0.95'] 
weatherVars  = ['precip_', 'temp_'] 
statVarsYr   = ['annualquant_', 'zipquant_', 'zipQuarterquant_']
outcomeVars  = ['lnRevNormd', 'lnCostNormd']


industries = range(1,44)


start = time.time()

results = pd.DataFrame()

i = 0

for ind in industries:
    print('##########################################################')
    print(ind)
    filename = '../../data/companyData/igData_ind' + str(ind) + '.csv'           
    goodsData = pd.read_csv(filename).drop(columns = {'Unnamed: 0'})

    
    for outcomeVar in outcomeVars:
        for weatherVar in weatherVars:
            for statVar in statVarsYr:                     
                for cutoffVar in cutoffVarsYr:
                    
                    i = i + 1
                    
                    
                    
                    '''goodsData = goodsData[~goodsData.lnRev.isna() & 
                                         ~goodsData.lnCost.isna() & 
                                         ~goodsData.revenueChange.isna() & 
                                         ~goodsData.costChange.isna()]'''
                    
                    
                    indVar = weatherVar + statVar + cutoffVar


                    print(outcomeVar, "~", indVar)


                    # find: concurrent ; or lagged supplier data
                    X = goodsData.loc[:,((goodsData.columns.str.contains(indVar) & ~goodsData.columns.str.contains('empWt_')) | 
                                                    (goodsData.columns.str.contains('indQtr_')) |
                                                    (goodsData.columns.str.contains('gvkey_'))  | 
                                                    (goodsData.columns.str.contains('ageTercile_')) |
                                                    (goodsData.columns.str.contains('sizeTercile_')) |
                                                    (goodsData.columns.str.contains('profitTercile_')))]


                    firms = goodsData['gvkey']


                    y = goodsData[outcomeVar]


                    model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                    pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar) & ~goodsData.columns.str.contains('empWt_')] )]
                    coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar) & ~goodsData.columns.str.contains('empWt_')])]
                    print(pvals)
                    # results.loc[i,'ind'] = ind

                    results.loc[i,'industry'] = ind

                    results.loc[i,'outcomeVar'] = outcomeVar
                    results.loc[i,'weatherVar'] = weatherVar
                    results.loc[i,'statVar']    = statVar
                    results.loc[i,'cutoffVar']  = cutoffVar
                    # results.loc[i,'firmVar']    = firmVar


                    results.loc[i,'lag0']       = coeff[0]
                    results.loc[i,'lag1']       = coeff[1]
                    results.loc[i,'lag2']       = coeff[2]
                    results.loc[i,'lag3']       = coeff[3]
                    results.loc[i,'lag4']       = coeff[4]

                    results.loc[i,'pval0']      = pvals[0]
                    results.loc[i,'pval1']      = pvals[1]
                    results.loc[i,'pval2']      = pvals[2]
                    results.loc[i,'pval3']      = pvals[3]
                    results.loc[i,'pval4']      = pvals[4]


                    results.to_csv("../../results_byInds_withControls_logNormd.csv")

                    print( time.time() - start)

##########################################################
1
lnRevNormd ~ precip_annualquant_0.95
precip_annualquant_0.95         0.002304
lag1_precip_annualquant_0.95    0.017745
lag2_precip_annualquant_0.95    0.043583
lag3_precip_annualquant_0.95    0.475367
lag4_precip_annualquant_0.95    0.008036
dtype: float64
0.06957101821899414
lnRevNormd ~ precip_zipquant_0.95
precip_zipquant_0.95         0.643767
lag1_precip_zipquant_0.95    0.009990
lag2_precip_zipquant_0.95    0.662644
lag3_precip_zipquant_0.95    0.519059
lag4_precip_zipquant_0.95    0.188017
dtype: float64
0.08039712905883789
lnRevNormd ~ precip_zipQuarterquant_0.95
precip_zipQuarterquant_0.95         0.254911
lag1_precip_zipQuarterquant_0.95    0.011101
lag2_precip_zipQuarterquant_0.95    0.415283
lag3_precip_zipQuarterquant_0.95    0.789969
lag4_precip_zipQuarterquant_0.95    0.039351
dtype: float64
0.09075307846069336
lnRevNormd ~ temp_annualquant_0.95
temp_annualquant_0.95         0.557377
lag1_temp_annualquant_0.95  

lnRevNormd ~ precip_annualquant_0.95
precip_annualquant_0.95         0.761896
lag1_precip_annualquant_0.95    0.057245
lag2_precip_annualquant_0.95    0.245967
lag3_precip_annualquant_0.95    0.018407
lag4_precip_annualquant_0.95    0.167738
dtype: float64
1.7438251972198486
lnRevNormd ~ precip_zipquant_0.95
precip_zipquant_0.95         0.629986
lag1_precip_zipquant_0.95    0.050658
lag2_precip_zipquant_0.95    0.172993
lag3_precip_zipquant_0.95    0.011835
lag4_precip_zipquant_0.95    0.175090
dtype: float64
1.7668390274047852
lnRevNormd ~ precip_zipQuarterquant_0.95
precip_zipQuarterquant_0.95         0.359975
lag1_precip_zipQuarterquant_0.95    0.034709
lag2_precip_zipQuarterquant_0.95    0.093834
lag3_precip_zipQuarterquant_0.95    0.014131
lag4_precip_zipQuarterquant_0.95    0.430249
dtype: float64
1.817167043685913
lnRevNormd ~ temp_annualquant_0.95
temp_annualquant_0.95         0.166797
lag1_temp_annualquant_0.95    0.257459
lag2_temp_annualquant_0.95    0.047434
lag3_temp_annua

precip_annualquant_0.95         0.033230
lag1_precip_annualquant_0.95    0.339022
lag2_precip_annualquant_0.95    0.021151
lag3_precip_annualquant_0.95    0.067282
lag4_precip_annualquant_0.95    0.569165
dtype: float64
2.416917085647583
lnCostNormd ~ precip_zipquant_0.95
precip_zipquant_0.95         0.080314
lag1_precip_zipquant_0.95    0.279784
lag2_precip_zipquant_0.95    0.286975
lag3_precip_zipquant_0.95    0.351590
lag4_precip_zipquant_0.95    0.930057
dtype: float64
2.4458413124084473
lnCostNormd ~ precip_zipQuarterquant_0.95
precip_zipQuarterquant_0.95         0.010985
lag1_precip_zipQuarterquant_0.95    0.079155
lag2_precip_zipQuarterquant_0.95    0.324229
lag3_precip_zipQuarterquant_0.95    0.353917
lag4_precip_zipQuarterquant_0.95    0.469716
dtype: float64
2.470355987548828
lnCostNormd ~ temp_annualquant_0.95
temp_annualquant_0.95         0.129061
lag1_temp_annualquant_0.95    0.999834
lag2_temp_annualquant_0.95    0.087589
lag3_temp_annualquant_0.95    0.191607
lag4_temp_a

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ precip_annualquant_0.95
precip_annualquant_0.95         0.024310
lag1_precip_annualquant_0.95    0.000639
lag2_precip_annualquant_0.95    0.002062
lag3_precip_annualquant_0.95    0.013082
lag4_precip_annualquant_0.95    0.114121
dtype: float64
2.875335216522217
lnRevNormd ~ precip_zipquant_0.95
precip_zipquant_0.95         0.076473
lag1_precip_zipquant_0.95    0.003424
lag2_precip_zipquant_0.95    0.019295
lag3_precip_zipquant_0.95    0.024507
lag4_precip_zipquant_0.95    0.236394
dtype: float64
2.9859509468078613
lnRevNormd ~ precip_zipQuarterquant_0.95
precip_zipQuarterquant_0.95         0.024984
lag1_precip_zipQuarterquant_0.95    0.002037
lag2_precip_zipQuarterquant_0.95    0.014522
lag3_precip_zipQuarterquant_0.95    0.026181
lag4_precip_zipQuarterquant_0.95    0.152402
dtype: float64
3.1013970375061035
lnRevNormd ~ temp_annualquant_0.95
temp_annualquant_0.95         0.235682
lag1_temp_annualquant_0.95    0.991482
lag2_temp_annualquant_0.95    0.650781
lag3_temp_annua

...

168.44583010673523
lnRevNormd ~ temp_zipquant_0.95
temp_zipquant_0.95         0.230405
lag1_temp_zipquant_0.95    0.588119
lag2_temp_zipquant_0.95    0.431699
lag3_temp_zipquant_0.95    0.412139
lag4_temp_zipquant_0.95    0.575756
dtype: float64
168.47488713264465
lnRevNormd ~ temp_zipQuarterquant_0.95
temp_zipQuarterquant_0.95         0.626656
lag1_temp_zipQuarterquant_0.95    0.733021
lag2_temp_zipQuarterquant_0.95    0.594433
lag3_temp_zipQuarterquant_0.95    0.985175
lag4_temp_zipQuarterquant_0.95    0.714984
dtype: float64
168.5042290687561
lnCostNormd ~ precip_annualquant_0.95
precip_annualquant_0.95         0.007267
lag1_precip_annualquant_0.95    0.086154
lag2_precip_annualquant_0.95    0.025446
lag3_precip_annualquant_0.95    0.007927
lag4_precip_annualquant_0.95    0.043642
dtype: float64
168.5372121334076
lnCostNormd ~ precip_zipquant_0.95
precip_zipquant_0.95         0.035809
lag1_precip_zipquant_0.95    0.086266
lag2_precip_zipquant_0.95    0.043401
lag3_precip_zipquant_0.

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ precip_annualquant_0.95
precip_annualquant_0.95         0.020969
lag1_precip_annualquant_0.95    0.018634
lag2_precip_annualquant_0.95    0.000481
lag3_precip_annualquant_0.95    0.000487
lag4_precip_annualquant_0.95    0.028433
dtype: float64
169.3811571598053
lnRevNormd ~ precip_zipquant_0.95
precip_zipquant_0.95         0.013622
lag1_precip_zipquant_0.95    0.035140
lag2_precip_zipquant_0.95    0.008265
lag3_precip_zipquant_0.95    0.000472
lag4_precip_zipquant_0.95    0.012390
dtype: float64
169.58132815361023
lnRevNormd ~ precip_zipQuarterquant_0.95
precip_zipQuarterquant_0.95         0.001486
lag1_precip_zipQuarterquant_0.95    0.050631
lag2_precip_zipQuarterquant_0.95    0.000292
lag3_precip_zipQuarterquant_0.95    0.000156
lag4_precip_zipQuarterquant_0.95    0.002084
dtype: float64
169.76934814453125
lnRevNormd ~ temp_annualquant_0.95
temp_annualquant_0.95         0.684932
lag1_temp_annualquant_0.95    0.124202
lag2_temp_annualquant_0.95    0.026068
lag3_temp_annua

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ precip_annualquant_0.95
precip_annualquant_0.95         0.584193
lag1_precip_annualquant_0.95    0.974707
lag2_precip_annualquant_0.95    0.686135
lag3_precip_annualquant_0.95    0.528147
lag4_precip_annualquant_0.95    0.670428
dtype: float64
172.46714329719543
lnRevNormd ~ precip_zipquant_0.95
precip_zipquant_0.95         0.061871
lag1_precip_zipquant_0.95    0.223434
lag2_precip_zipquant_0.95    0.099543
lag3_precip_zipquant_0.95    0.031559
lag4_precip_zipquant_0.95    0.204764
dtype: float64
172.77867317199707
lnRevNormd ~ precip_zipQuarterquant_0.95
precip_zipQuarterquant_0.95         0.146894
lag1_precip_zipQuarterquant_0.95    0.540653
lag2_precip_zipQuarterquant_0.95    0.073159
lag3_precip_zipQuarterquant_0.95    0.113138
lag4_precip_zipQuarterquant_0.95    0.098740
dtype: float64
173.13498711585999
lnRevNormd ~ temp_annualquant_0.95
temp_annualquant_0.95         0.058233
lag1_temp_annualquant_0.95    0.314393
lag2_temp_annualquant_0.95    0.290725
lag3_temp_annu

## Faster and More Heuristic

The below gives us unclustered standard errors, output to a csv file.

In [ ]:
def findSE(X,reg,y):
    N = len(X)
    p = len(X.columns) + 1  # plus one because LinearRegression adds an intercept term

    X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
    X_with_intercept[:, 0] = 1
    X_with_intercept[:, 1:p] = X.values

    y_hat = reg.predict(X)
    residuals = y.values - y_hat
    residual_sum_of_squares = residuals.T @ residuals
    sigma_squared_hat = residual_sum_of_squares / (N - p)
    var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat

    se0 = var_beta_hat[1, 1] ** 0.5
    se1 = var_beta_hat[2, 2] ** 0.5
    se2 = var_beta_hat[3, 3] ** 0.5
    se3 = var_beta_hat[4, 4] ** 0.5
    se4 = var_beta_hat[5, 5] ** 0.5
    se5 = var_beta_hat[6, 6] ** 0.5
    '''se6 = var_beta_hat[7, 7] ** 0.5
    se7 = var_beta_hat[8, 8] ** 0.5
    se8 = var_beta_hat[9, 9] ** 0.5'''
    return([abs(reg.coef_[0]/se0),abs(reg.coef_[1]/se1),abs(reg.coef_[2]/se2),
            abs(reg.coef_[3]/se3),abs(reg.coef_[4]/se4),abs(reg.coef_[5]/se5)]
          )

'''        
abs(reg.coef_[0]/se0),
          abs(reg.coef_[1]/se1),
          abs(reg.coef_[2]/se2),
          abs(reg.coef_[3]/se3),
          abs(reg.coef_[4]/se4),
          abs(reg.coef_[5]/se5),
          "SE0: ", se0,
          "SE1: ", se1,
          "SE2: ", se2,
          "SE3: ", se3,
          "SE4: ", se4,
          "SE5: ", se5,

'''


In [ ]:
goodsData.columns[0:75]

In [ ]:
goodsData.columns[0:50]

In [ ]:
inds = [1, 2, 6, 7, 18, 31, 41, 42]

for ind in inds:

    tempData = goodsData[goodsData.famafrench == ind]
    print(tempData.shape)


In [ ]:
'''cutoffVarsYr = ['0.95'] # ,'1xYr']                                    #,'1x5Yrs'] #, ] # ,'1xQtr', '1x5Qtrs'
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['zipquant_','zipQuarterquant_']
outcomeVars  = ['lnRev', 'revenueChange'] # ,'lnCost',  'costChange'] # [,'lnRevNormd','lnCostNormd'] # 'revenueChange' 'costChange',
firmVars     = ['firmQtr_'] # 'gvkey'
'''

# try this by industry
cutoffVarsYr = ['0.95'] # ,'1xYr']                                    #,'1x5Yrs'] #, ] # ,'1xQtr', '1x5Qtrs'
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['ffquant_','indQuarterquant_']
outcomeVars  = ['lnRev', 'revenueChange',  'lnCost',  'costChange'] # [,'lnRevNormd','lnCostNormd'] # 'revenueChange' 'costChange',
firmVars     = ['firmQtr_']


inds = [1, 2, 6, 7, 18, 31, 41, 42]

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() &
                      ~goodsData.lnCostNormd.isna() & ~goodsData.lnRevNormd.isna()]

start = time.time()

results = pd.DataFrame()
i = 0
for ind in inds:
    print('#######################################################################################',ind)
    for outcomeVar in outcomeVars:
        for weatherVar in weatherVars:
            for statVar in statVarsYr:                     
                for cutoffVar in cutoffVarsYr:
                    for firmVar in firmVars:
                        tempData = goodsData[goodsData.famafrench == ind]
                        
                        i = i + 1
                        indVar = weatherVar + statVar + cutoffVar


                        print(outcomeVar, "~", indVar, "|", firmVar)


                        # find: concurrent ; or lagged supplier data
                        X = tempData.loc[:,((tempData.columns.str.contains(indVar)) |
                                          (tempData.columns.str.contains('indQtr_')) |
                                          # (goodsData.columns.str.contains('gvkey_'))) |   # &   
                                          # (goodsData.columns.str.contains('firmQtr_'))) |
                                          (tempData.columns.str.contains(firmVar)))] # |
                        '''(tempData.columns.str.contains('ageQtr_')) |
                          (tempData.columns.str.contains('sizeQtr_')) |
                          (tempData.columns.str.contains('profitQtr_'))]   #  & '''

                                          # (goodsData.columns.str.contains('firmQtr_')))       & 
                                        # ~(goodsData.columns.str.contains('lag4')) &
                                                                        # ~(goodsData.columns.str.contains('lag2')) & 


                        X = X[X.columns[(X.sum(axis = 0) >= 4)]]
                        # print(X.columns)
                        firms = tempData['gvkey']


                        y = tempData[outcomeVar]


                        ######################################
                        # fit the model on this subset
                        reg = linear_model.LinearRegression()
                        reg.fit(X,y)


                        # print('Coeff: ' , reg.coef_[0:5], 'SE type (looking >2): ', findSE(X,reg,y))
                        results.loc[i,'ind'] = ind


                        results.loc[i,'outcomeVar'] = outcomeVar
                        results.loc[i,'weatherVar'] = weatherVar
                        results.loc[i,'statVar']    = statVar
                        results.loc[i,'cutoffVar']  = cutoffVar
                        results.loc[i,'firmVar']    = firmVar


                        results.loc[i,'lag0']       = reg.coef_[0]
                        results.loc[i,'lag1']       = reg.coef_[1]
                        results.loc[i,'lag2']       = reg.coef_[2]
                        results.loc[i,'lag3']       = reg.coef_[3]
                        results.loc[i,'lag4']       = reg.coef_[4]



                        seratios = findSE(X,reg,y)

                        results.loc[i,'ratio0']       = seratios[0]
                        results.loc[i,'ratio1']       = seratios[1]
                        results.loc[i,'ratio2']       = seratios[2]
                        results.loc[i,'ratio3']       = seratios[3]
                        results.loc[i,'ratio4']       = seratios[4]

                        # print(results)

                        print(time.time() - start)

                        print('*******************************************************************')
                    
results.to_csv("../../data/results_notNormd.csv")

In [ ]:
i = 0

results = pd.DataFrame()
results.loc[i,'outcome'] = 0
results

# Indirect Effects
This is almost exactly the same but with supplier information in place of the direct company information.

In [41]:
os.getcwd()

'/Users/brianreed/Documents/supplyChain/extremes/extremesAnalysisCode'

In [ ]:
filename = "../../data/companyData/supplier_igData_ind" + str(ind) + ".csv"
goodsData = pd.read_csv(filename).drop(columns = {'Unnamed: 0'})



In [101]:
cutoffVarsYr = ['0.95'] 
weatherVars  = ['precip_', 'temp_'] 
statVarsYr   = ['zipQuarterquant_']
outcomeVars  = ['lnRevNormd', 'lnCostNormd']


industries = range(1,44)


start = time.time()

results = pd.DataFrame()

i = 0



for ind in industries:
    print('##########################################################')
    print(ind)
    
    filename = "../../data/companyData/supplier_igData_ind" + str(ind) + ".csv"
    goodsData = pd.read_csv(filename).drop(columns = {'Unnamed: 0'})

    if goodsData.shape[0] > 50:
        for outcomeVar in outcomeVars:
            for weatherVar in weatherVars:
                for statVar in statVarsYr:                     
                    for cutoffVar in cutoffVarsYr:

                        i = i + 1



                        '''goodsData = goodsData[~goodsData.lnRev.isna() & 
                                             ~goodsData.lnCost.isna() & 
                                             ~goodsData.revenueChange.isna() & 
                                             ~goodsData.costChange.isna()]'''


                        indVar = weatherVar + statVar + cutoffVar


                        print(outcomeVar, "~", indVar)


                        # find: concurrent ; or lagged supplier data
                        X = goodsData.loc[:,(((goodsData.columns.str.contains(indVar)) & ~goodsData.columns.str.contains('empWt_')) | 
                                (goodsData.columns.str.contains('indQtr_')) |
                                (goodsData.columns.str.contains('gvkey_')) | #  | 
                                (goodsData.columns.str.contains('ageTercile_')) |
                                (goodsData.columns.str.contains('sizeTercile_')) |
                                (goodsData.columns.str.contains('profitTercile_')) | 
                                (goodsData.columns == 'supplierTercile'))] 

                        print(X.columns)
                        firms = goodsData['gvkey']


                        y = goodsData[outcomeVar]


                        model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                        pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar) & ~goodsData.columns.str.contains('empWt_')] )]
                        coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar) & ~goodsData.columns.str.contains('empWt_')])]
                        print(pvals)
                        # results.loc[i,'ind'] = ind

                        results.loc[i,'industry'] = ind

                        results.loc[i,'outcomeVar'] = outcomeVar
                        results.loc[i,'weatherVar'] = weatherVar
                        results.loc[i,'statVar']    = statVar
                        results.loc[i,'cutoffVar']  = cutoffVar
                        # results.loc[i,'firmVar']    = firmVar


                        results.loc[i,'lag0']       = coeff[0]
                        results.loc[i,'lag1']       = coeff[1]
                        results.loc[i,'lag2']       = coeff[2]
                        results.loc[i,'lag3']       = coeff[3]
                        results.loc[i,'lag4']       = coeff[4]

                        results.loc[i,'pval0']      = pvals[0]
                        results.loc[i,'pval1']      = pvals[1]
                        results.loc[i,'pval2']      = pvals[2]
                        results.loc[i,'pval3']      = pvals[3]
                        results.loc[i,'pval4']      = pvals[4]


                        results.to_csv("../../indirResults.csv")

                        print( time.time() - start)

##########################################################
1
##########################################################
2
lnRevNormd ~ precip_zipQuarterquant_0.95
Index(['supplier_precip_zipQuarterquant_0.95',
       'supplier_lag1_precip_zipQuarterquant_0.95',
       'supplier_lag2_precip_zipQuarterquant_0.95',
       'supplier_lag3_precip_zipQuarterquant_0.95',
       'supplier_lag4_precip_zipQuarterquant_0.95', 'gvkey_8479',
       'gvkey_10247', 'gvkey_13540', 'gvkey_18312', 'gvkey_20747',
       'gvkey_24893', 'gvkey_158743', 'ageTercile_2', 'ageTercile_3',
       'sizeTercile_2', 'sizeTercile_3', 'profitTercile_2', 'profitTercile_3'],
      dtype='object')
supplier_precip_zipQuarterquant_0.95         0.045126
supplier_lag1_precip_zipQuarterquant_0.95    0.566523
supplier_lag2_precip_zipQuarterquant_0.95    0.142475
supplier_lag3_precip_zipQuarterquant_0.95    0.240356
supplier_lag4_precip_zipQuarterquant_0.95    0.417212
dtype: float64
0.0840301513671875
lnRevNormd ~ temp_zipQuar

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ precip_zipQuarterquant_0.95
Index(['supplier_precip_zipQuarterquant_0.95',
       'supplier_lag1_precip_zipQuarterquant_0.95',
       'supplier_lag2_precip_zipQuarterquant_0.95',
       'supplier_lag3_precip_zipQuarterquant_0.95',
       'supplier_lag4_precip_zipQuarterquant_0.95', 'gvkey_1602', 'gvkey_1794',
       'gvkey_2086', 'gvkey_2220', 'gvkey_2403',
       ...
       'gvkey_176361', 'gvkey_178201', 'gvkey_178972', 'gvkey_179027',
       'ageTercile_2', 'ageTercile_3', 'sizeTercile_2', 'sizeTercile_3',
       'profitTercile_2', 'profitTercile_3'],
      dtype='object', length=127)
supplier_precip_zipQuarterquant_0.95         0.277958
supplier_lag1_precip_zipQuarterquant_0.95    0.268788
supplier_lag2_precip_zipQuarterquant_0.95    0.758762
supplier_lag3_precip_zipQuarterquant_0.95    0.460504
supplier_lag4_precip_zipQuarterquant_0.95    0.446917
dtype: float64
1.7873449325561523
lnRevNormd ~ temp_zipQuarterquant_0.95
Index(['supplier_temp_zipQuarterquant_0.95',
    

supplier_precip_zipQuarterquant_0.95         0.007215
supplier_lag1_precip_zipQuarterquant_0.95    0.000654
supplier_lag2_precip_zipQuarterquant_0.95    0.000487
supplier_lag3_precip_zipQuarterquant_0.95    0.000683
supplier_lag4_precip_zipQuarterquant_0.95    0.000113
dtype: float64
3.9940850734710693
lnRevNormd ~ temp_zipQuarterquant_0.95
Index(['supplier_temp_zipQuarterquant_0.95',
       'supplier_lag1_temp_zipQuarterquant_0.95',
       'supplier_lag2_temp_zipQuarterquant_0.95',
       'supplier_lag3_temp_zipQuarterquant_0.95',
       'supplier_lag4_temp_zipQuarterquant_0.95', 'gvkey_2771', 'gvkey_2991',
       'gvkey_4503', 'gvkey_5046', 'gvkey_5047', 'gvkey_5229', 'gvkey_6774',
       'gvkey_7985', 'gvkey_8253', 'gvkey_8539', 'gvkey_8762', 'gvkey_9465',
       'gvkey_10622', 'gvkey_11217', 'gvkey_16101', 'gvkey_17436',
       'gvkey_29156', 'gvkey_29392', 'gvkey_29968', 'gvkey_30170',
       'gvkey_64768', 'gvkey_65350', 'gvkey_100080', 'ageTercile_2',
       'ageTercile_3', 'siz

      dtype='object')
supplier_precip_zipQuarterquant_0.95         0.182193
supplier_lag1_precip_zipQuarterquant_0.95    0.196697
supplier_lag2_precip_zipQuarterquant_0.95    0.240819
supplier_lag3_precip_zipQuarterquant_0.95    0.241996
supplier_lag4_precip_zipQuarterquant_0.95    0.220818
dtype: float64
4.449359893798828
lnRevNormd ~ temp_zipQuarterquant_0.95
Index(['supplier_temp_zipQuarterquant_0.95',
       'supplier_lag1_temp_zipQuarterquant_0.95',
       'supplier_lag2_temp_zipQuarterquant_0.95',
       'supplier_lag3_temp_zipQuarterquant_0.95',
       'supplier_lag4_temp_zipQuarterquant_0.95', 'gvkey_2136', 'gvkey_2176',
       'gvkey_2410', 'gvkey_2884', 'gvkey_2991', 'gvkey_3226', 'gvkey_4029',
       'gvkey_4093', 'gvkey_4242', 'gvkey_4430', 'gvkey_4503', 'gvkey_4517',
       'gvkey_5742', 'gvkey_6135', 'gvkey_7366', 'gvkey_7733', 'gvkey_7970',
       'gvkey_7974', 'gvkey_7977', 'gvkey_8099', 'gvkey_8264', 'gvkey_8272',
       'gvkey_8539', 'gvkey_9846', 'gvkey_9850', 'gvkey

...

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ precip_zipQuarterquant_0.95
Index(['supplier_precip_zipQuarterquant_0.95',
       'supplier_lag1_precip_zipQuarterquant_0.95',
       'supplier_lag2_precip_zipQuarterquant_0.95',
       'supplier_lag3_precip_zipQuarterquant_0.95',
       'supplier_lag4_precip_zipQuarterquant_0.95', 'gvkey_1300', 'gvkey_1327',
       'gvkey_1690', 'gvkey_1773', 'gvkey_1913',
       ...
       'gvkey_176404', 'gvkey_177257', 'gvkey_177486', 'gvkey_179202',
       'ageTercile_2', 'ageTercile_3', 'sizeTercile_2', 'sizeTercile_3',
       'profitTercile_2', 'profitTercile_3'],
      dtype='object', length=169)
supplier_precip_zipQuarterquant_0.95         0.223670
supplier_lag1_precip_zipQuarterquant_0.95    0.007157
supplier_lag2_precip_zipQuarterquant_0.95    0.610744
supplier_lag3_precip_zipQuarterquant_0.95    0.352271
supplier_lag4_precip_zipQuarterquant_0.95    0.681684
dtype: float64
11.37671685218811
lnRevNormd ~ temp_zipQuarterquant_0.95
Index(['supplier_temp_zipQuarterquant_0.95',
     

supplier_precip_zipQuarterquant_0.95         0.064179
supplier_lag1_precip_zipQuarterquant_0.95    0.094653
supplier_lag2_precip_zipQuarterquant_0.95    0.201308
supplier_lag3_precip_zipQuarterquant_0.95    0.156359
supplier_lag4_precip_zipQuarterquant_0.95    0.180570
dtype: float64
14.343944072723389
lnCostNormd ~ temp_zipQuarterquant_0.95
Index(['supplier_temp_zipQuarterquant_0.95',
       'supplier_lag1_temp_zipQuarterquant_0.95',
       'supplier_lag2_temp_zipQuarterquant_0.95',
       'supplier_lag3_temp_zipQuarterquant_0.95',
       'supplier_lag4_temp_zipQuarterquant_0.95', 'gvkey_1690', 'gvkey_1919',
       'gvkey_2285', 'gvkey_2721', 'gvkey_2817', 'gvkey_3158', 'gvkey_4274',
       'gvkey_4321', 'gvkey_4439', 'gvkey_4839', 'gvkey_5047', 'gvkey_5073',
       'gvkey_5492', 'gvkey_5650', 'gvkey_5691', 'gvkey_5860', 'gvkey_6008',
       'gvkey_6268', 'gvkey_6494', 'gvkey_6565', 'gvkey_6774', 'gvkey_7114',
       'gvkey_7585', 'gvkey_7985', 'gvkey_8264', 'gvkey_8694', 'gvkey_8972'

supplier_temp_zipQuarterquant_0.95         0.127816
supplier_lag1_temp_zipQuarterquant_0.95    0.308606
supplier_lag2_temp_zipQuarterquant_0.95    0.359132
supplier_lag3_temp_zipQuarterquant_0.95    0.235222
supplier_lag4_temp_zipQuarterquant_0.95    0.525361
dtype: float64
14.78056001663208
lnCostNormd ~ precip_zipQuarterquant_0.95
Index(['supplier_precip_zipQuarterquant_0.95',
       'supplier_lag1_precip_zipQuarterquant_0.95',
       'supplier_lag2_precip_zipQuarterquant_0.95',
       'supplier_lag3_precip_zipQuarterquant_0.95',
       'supplier_lag4_precip_zipQuarterquant_0.95', 'gvkey_1230', 'gvkey_1380',
       'gvkey_1382', 'gvkey_2410', 'gvkey_2991', 'gvkey_3144', 'gvkey_3851',
       'gvkey_4060', 'gvkey_4503', 'gvkey_4839', 'gvkey_5073', 'gvkey_8068',
       'gvkey_8549', 'gvkey_10466', 'gvkey_10484', 'gvkey_10795',
       'gvkey_10920', 'gvkey_11506', 'gvkey_11923', 'gvkey_12384',
       'gvkey_13312', 'gvkey_14087', 'gvkey_14590', 'gvkey_18498',
       'gvkey_27786', 'gvkey

15.260040998458862
lnCostNormd ~ temp_zipQuarterquant_0.95
Index(['supplier_temp_zipQuarterquant_0.95',
       'supplier_lag1_temp_zipQuarterquant_0.95',
       'supplier_lag2_temp_zipQuarterquant_0.95',
       'supplier_lag3_temp_zipQuarterquant_0.95',
       'supplier_lag4_temp_zipQuarterquant_0.95', 'gvkey_1690', 'gvkey_1919',
       'gvkey_1932', 'gvkey_2111', 'gvkey_2136', 'gvkey_2220', 'gvkey_2285',
       'gvkey_2410', 'gvkey_2991', 'gvkey_3144', 'gvkey_3469', 'gvkey_3708',
       'gvkey_4060', 'gvkey_4213', 'gvkey_4503', 'gvkey_4510', 'gvkey_4839',
       'gvkey_5047', 'gvkey_5525', 'gvkey_5968', 'gvkey_6266', 'gvkey_6375',
       'gvkey_6774', 'gvkey_7171', 'gvkey_7186', 'gvkey_7241', 'gvkey_7985',
       'gvkey_8068', 'gvkey_8247', 'gvkey_8543', 'gvkey_8549', 'gvkey_8972',
       'gvkey_9899', 'gvkey_10984', 'gvkey_11923', 'gvkey_12141',
       'gvkey_12384', 'gvkey_12892', 'gvkey_13683', 'gvkey_14078',
       'gvkey_14934', 'gvkey_15070', 'gvkey_15247', 'gvkey_15334',
      

Now do this by streaks - consecutive days with at least 95th percentile temp or rain.

In [98]:
weatherVars  = ['hotStreak',  'wetStreak']   
outcomeVars  = ['lnRevNormd', 'lnCostNormd'] 

goodsData['scTercile']  = pd.qcut(goodsData['suppliers'], 3, labels=False, duplicates = 'drop')

results = pd.DataFrame()
i = 0

filename = "../../data/companyData/goodsData_supplierData.csv"
goodsData = pd.read_csv(filename).drop(columns = {'Unnamed: 0'})



for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:

        i = i + 1

        indVar = weatherVar


        print(outcomeVar, "~", indVar)


        # find: concurrent ; or lagged supplier datawet
        X = goodsData.loc[:,(((goodsData.columns.str.contains(indVar))) | 
                        (goodsData.columns.str.contains('indQtr_')) |
                        (goodsData.columns.str.contains('gvkey_')) | #  | 
                        (goodsData.columns.str.contains('ageTercile_')) |
                        (goodsData.columns.str.contains('sizeTercile_')) |
                        (goodsData.columns.str.contains('profitTercile_')) | 
                        (goodsData.columns == 'supplierTercile'))]     


        firms = goodsData['gvkey']


        y = goodsData[outcomeVar]


        modelResults = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
        pvals = modelResults.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('supplier_')])]
        coeff = modelResults.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)  & goodsData.columns.str.contains('supplier_')])]
        print(coeff)
        print(pvals)


        results.loc[i,'industry'] = ind

        results.loc[i,'outcomeVar'] = outcomeVar
        results.loc[i,'weatherVar'] = weatherVar

        results.loc[i,'lag0']       = coeff[0]
        results.loc[i,'lag1']       = coeff[1]
        results.loc[i,'lag2']       = coeff[2]
        results.loc[i,'lag3']       = coeff[3]
        results.loc[i,'lag4']       = coeff[4]

        results.loc[i,'pval0']      = pvals[0]
        results.loc[i,'pval1']      = pvals[1]
        results.loc[i,'pval2']      = pvals[2]
        results.loc[i,'pval3']      = pvals[3]
        results.loc[i,'pval4']      = pvals[4]



        print(time.time() - start)


results.to_csv("../../data/results_indirEffects_allGoodsData.csv")


/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(41481, 2935)
lnRevNormd ~ hotStreak
supplier_hotStreak         0.003296
supplier_lag1_hotStreak    0.000451
supplier_lag2_hotStreak    0.002933
supplier_lag3_hotStreak    0.004055
supplier_lag4_hotStreak    0.002615
dtype: float64
supplier_hotStreak         0.165592
supplier_lag1_hotStreak    0.864076
supplier_lag2_hotStreak    0.248385
supplier_lag3_hotStreak    0.171946
supplier_lag4_hotStreak    0.431067
dtype: float64
3523.670075893402
lnRevNormd ~ wetStreak
supplier_wetStreak         0.000865
supplier_lag1_wetStreak   -0.002419
supplier_lag2_wetStreak    0.002135
supplier_lag3_wetStreak   -0.004973
supplier_lag4_wetStreak   -0.005436
dtype: float64
supplier_wetStreak         0.999998
supplier_lag1_wetStreak    0.999998
supplier_lag2_wetStreak    0.999997
supplier_lag3_wetStreak    0.999995
supplier_lag4_wetStreak    0.999988
dtype: float64
3636.295879840851
lnCostNormd ~ hotStreak
supplier_hotStreak         0.003381
supplier_lag1_hotStreak    0.002548
supplier_lag2_hotStreak   -0

In [94]:
weatherVars  = ['hotStreak',  'wetStreak']   #[,]
outcomeVars  = ['lnRevNormd', 'lnCostNormd'] # ['revenueChange'] #[, 'costChange']#,'lnCost','lnInc','lnRev']

# goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() & ~goodsData.lnCostNormd.isna()]
goodsData['scTercile']  = pd.qcut(goodsData['suppliers'], 3, labels=False, duplicates = 'drop')


start = time.time()
results = pd.DataFrame()
i = 0

industries = range(1,44)

for ind in industries:
    filename = "../../data/companyData/supplier_igData_ind" + str(ind) + ".csv"
    goodsData = pd.read_csv(filename).drop(columns = {'Unnamed: 0'})

    if goodsData.shape[0] > 50:

        for outcomeVar in outcomeVars:
            for weatherVar in weatherVars:
                
                i = i + 1
                
                indVar = weatherVar


                print(outcomeVar, "~", indVar)


                # find: concurrent ; or lagged supplier datawet
                X = goodsData.loc[:,(((goodsData.columns.str.contains(indVar))) | 
                                (goodsData.columns.str.contains('indQtr_')) |
                                (goodsData.columns.str.contains('gvkey_')) | #  | 
                                (goodsData.columns.str.contains('ageTercile_')) |
                                (goodsData.columns.str.contains('sizeTercile_')) |
                                (goodsData.columns.str.contains('profitTercile_')) | 
                                (goodsData.columns == 'supplierTercile'))]     


                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]


                modelResults = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                pvals = modelResults.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('supplier_')])]
                coeff = modelResults.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)  & goodsData.columns.str.contains('supplier_')])]
                print(coeff)
                print(pvals)


                results.loc[i,'industry'] = ind

                results.loc[i,'outcomeVar'] = outcomeVar
                results.loc[i,'weatherVar'] = weatherVar

                results.loc[i,'lag0']       = coeff[0]
                results.loc[i,'lag1']       = coeff[1]
                results.loc[i,'lag2']       = coeff[2]
                results.loc[i,'lag3']       = coeff[3]
                results.loc[i,'lag4']       = coeff[4]

                results.loc[i,'pval0']      = pvals[0]
                results.loc[i,'pval1']      = pvals[1]
                results.loc[i,'pval2']      = pvals[2]
                results.loc[i,'pval3']      = pvals[3]
                results.loc[i,'pval4']      = pvals[4]
                
                
                
                print( time.time() - start)

                results.to_csv("../../data/results_indirEffects.csv")



lnRevNormd ~ hotStreak
supplier_hotStreak        -0.104862
supplier_lag1_hotStreak   -0.152431
supplier_lag2_hotStreak   -0.156054
supplier_lag3_hotStreak   -0.051794
supplier_lag4_hotStreak   -0.044449
dtype: float64
supplier_hotStreak         0.158455
supplier_lag1_hotStreak    0.075131
supplier_lag2_hotStreak    0.073915
supplier_lag3_hotStreak    0.261475
supplier_lag4_hotStreak    0.044471
dtype: float64
0.07126188278198242
lnRevNormd ~ wetStreak
supplier_wetStreak         0.129891
supplier_lag1_wetStreak    0.188396
supplier_lag2_wetStreak    0.321053
supplier_lag3_wetStreak   -0.168901
supplier_lag4_wetStreak   -0.000771
dtype: float64
supplier_wetStreak         0.244808
supplier_lag1_wetStreak    0.108118
supplier_lag2_wetStreak    0.016424
supplier_lag3_wetStreak    0.040185
supplier_lag4_wetStreak    0.990558
dtype: float64
0.07990288734436035
lnCostNormd ~ hotStreak
supplier_hotStreak        -0.141092
supplier_lag1_hotStreak   -0.204573
supplier_lag2_hotStreak   -0.205773
su

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ hotStreak
supplier_hotStreak        -0.023392
supplier_lag1_hotStreak   -0.028906
supplier_lag2_hotStreak   -0.034305
supplier_lag3_hotStreak   -0.021971
supplier_lag4_hotStreak   -0.024443
dtype: float64
supplier_hotStreak         0.269844
supplier_lag1_hotStreak    0.067309
supplier_lag2_hotStreak    0.009023
supplier_lag3_hotStreak    0.210936
supplier_lag4_hotStreak    0.049330
dtype: float64
2.241576910018921
lnRevNormd ~ wetStreak
supplier_wetStreak         0.031445
supplier_lag1_wetStreak    0.016341
supplier_lag2_wetStreak    0.049575
supplier_lag3_wetStreak    0.032163
supplier_lag4_wetStreak    0.062861
dtype: float64
supplier_wetStreak         0.066635
supplier_lag1_wetStreak    0.421608
supplier_lag2_wetStreak    0.071451
supplier_lag3_wetStreak    0.059414
supplier_lag4_wetStreak    0.004845
dtype: float64
3.691999912261963
lnCostNormd ~ hotStreak
supplier_hotStreak        -0.037988
supplier_lag1_hotStreak   -0.033981
supplier_lag2_hotStreak   -0.041011
suppli

lnRevNormd ~ hotStreak
supplier_hotStreak        -0.091394
supplier_lag1_hotStreak   -0.067668
supplier_lag2_hotStreak   -0.100613
supplier_lag3_hotStreak   -0.055473
supplier_lag4_hotStreak   -0.092453
dtype: float64
supplier_hotStreak         0.052769
supplier_lag1_hotStreak    0.166166
supplier_lag2_hotStreak    0.039199
supplier_lag3_hotStreak    0.186712
supplier_lag4_hotStreak    0.107213
dtype: float64
6.383235931396484
lnRevNormd ~ wetStreak
supplier_wetStreak        -0.023664
supplier_lag1_wetStreak   -0.012898
supplier_lag2_wetStreak    0.004446
supplier_lag3_wetStreak   -0.039733
supplier_lag4_wetStreak   -0.030871
dtype: float64
supplier_wetStreak         0.539323
supplier_lag1_wetStreak    0.753668
supplier_lag2_wetStreak    0.909470
supplier_lag3_wetStreak    0.253870
supplier_lag4_wetStreak    0.242759
dtype: float64
6.432568788528442
lnCostNormd ~ hotStreak
supplier_hotStreak        -0.085661
supplier_lag1_hotStreak   -0.077605
supplier_lag2_hotStreak   -0.096402
suppli

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ hotStreak
supplier_hotStreak        -0.016611
supplier_lag1_hotStreak   -0.016360
supplier_lag2_hotStreak   -0.036780
supplier_lag3_hotStreak   -0.030727
supplier_lag4_hotStreak   -0.025317
dtype: float64
supplier_hotStreak         0.310910
supplier_lag1_hotStreak    0.252468
supplier_lag2_hotStreak    0.053778
supplier_lag3_hotStreak    0.183772
supplier_lag4_hotStreak    0.115997
dtype: float64
12.018733739852905
lnRevNormd ~ wetStreak
supplier_wetStreak        -0.034645
supplier_lag1_wetStreak   -0.034387
supplier_lag2_wetStreak   -0.059050
supplier_lag3_wetStreak   -0.113367
supplier_lag4_wetStreak   -0.084556
dtype: float64
supplier_wetStreak         0.171435
supplier_lag1_wetStreak    0.276635
supplier_lag2_wetStreak    0.004461
supplier_lag3_wetStreak    0.071948
supplier_lag4_wetStreak    0.112172
dtype: float64
12.067215919494629
lnCostNormd ~ hotStreak
supplier_hotStreak        -0.018493
supplier_lag1_hotStreak   -0.017053
supplier_lag2_hotStreak   -0.049850
supp

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ hotStreak
supplier_hotStreak        -0.006775
supplier_lag1_hotStreak   -0.019928
supplier_lag2_hotStreak   -0.017071
supplier_lag3_hotStreak    0.002783
supplier_lag4_hotStreak   -0.012222
dtype: float64
supplier_hotStreak         0.414872
supplier_lag1_hotStreak    0.024700
supplier_lag2_hotStreak    0.018538
supplier_lag3_hotStreak    0.700312
supplier_lag4_hotStreak    0.112516
dtype: float64
13.756940126419067
lnRevNormd ~ wetStreak
supplier_wetStreak        -0.012094
supplier_lag1_wetStreak    0.004077
supplier_lag2_wetStreak    0.035659
supplier_lag3_wetStreak    0.042771
supplier_lag4_wetStreak    0.027651
dtype: float64
supplier_wetStreak         0.564635
supplier_lag1_wetStreak    0.818332
supplier_lag2_wetStreak    0.103429
supplier_lag3_wetStreak    0.022998
supplier_lag4_wetStreak    0.142140
dtype: float64
14.507429838180542
lnCostNormd ~ hotStreak
supplier_hotStreak        -0.007190
supplier_lag1_hotStreak   -0.018938
supplier_lag2_hotStreak   -0.023379
supp

supplier_hotStreak        -0.064258
supplier_lag1_hotStreak   -0.071849
supplier_lag2_hotStreak   -0.054919
supplier_lag3_hotStreak   -0.033532
supplier_lag4_hotStreak   -0.011328
dtype: float64
supplier_hotStreak         0.002882
supplier_lag1_hotStreak    0.000185
supplier_lag2_hotStreak    0.003626
supplier_lag3_hotStreak    0.077922
supplier_lag4_hotStreak    0.588240
dtype: float64
17.03663182258606
lnRevNormd ~ wetStreak
supplier_wetStreak         0.025384
supplier_lag1_wetStreak    0.001278
supplier_lag2_wetStreak   -0.024608
supplier_lag3_wetStreak   -0.126644
supplier_lag4_wetStreak   -0.074675
dtype: float64
supplier_wetStreak         0.265990
supplier_lag1_wetStreak    0.976815
supplier_lag2_wetStreak    0.614608
supplier_lag3_wetStreak    0.105233
supplier_lag4_wetStreak    0.100144
dtype: float64
17.115436792373657
lnCostNormd ~ hotStreak
supplier_hotStreak        -0.067033
supplier_lag1_hotStreak   -0.073969
supplier_lag2_hotStreak   -0.079460
supplier_lag3_hotStreak   -0